In [705]:
%pip install ortools
import networkx as nx
import random
from ortools.linear_solver import pywraplp

# Configurações gerais
n = 100  # número de vértices
p = 0.5  # probabilidade de aresta
random.seed(42)

# Gerando o grafo aleatório
G = nx.gnp_random_graph(n, p)
for (u, v) in G.edges():
    G.edges[u, v]['weight'] = random.randint(10, 500)

# Visualização básica (opcional)
print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())


Note: you may need to restart the kernel to use updated packages.
Número de vértices: 100
Número de arestas: 2449


In [706]:
import itertools
import time
def solve_with_subtour_elimination(G, d):
    # Criando o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variáveis de decisão: x[u, v] = 1 se a aresta (u, v) é selecionada, 0 caso contrário
    x = {}
    for u, v in G.edges():
        x[u, v] = solver.BoolVar(f'x_{u}_{v}')
        x[v, u] = solver.BoolVar(f'x_{v}_{u}')  # Considerando grafo não direcionado

    # Função objetivo: minimizar o custo das arestas selecionadas
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * x[u, v] for u, v in G.edges()))

    # 1. Restrição de árvore geradora no subgrafo dos centrais: 
    centrais = d.keys()
    solver.Add(solver.Sum(x[u, v] for u, v in G.edges() if u in centrais and v in centrais) == len(centrais) - 1)

    # 2. Eliminação de ciclos (Subtour Elimination Constraints - SECs) no subgrafo dos centrais
    for subset in itertools.chain.from_iterable(itertools.combinations(centrais, r) for r in range(2, len(centrais))):
        solver.Add(solver.Sum(x[u, v] for u, v in itertools.combinations(subset, 2) if G.has_edge(u, v)) <= len(subset) - 1)

    # 3. Restrição de grau mínimo para os vértices centrais
    for v in centrais:
        solver.Add(solver.Sum(x[u, v] for u in G.neighbors(v)) >= d[v])

    # 4. Cada terminal deve ser conectado a apenas um vértice central (grau 1)
    terminals = set(G.nodes()) - set(centrais)  # Supondo que vértices não em 'd' são terminais
    for t in terminals:
        solver.Add(solver.Sum(x[t, v] for v in G.neighbors(t)) == 1)

    # 5. Garantir que as variáveis são binárias (0 ou 1)
    for u, v in G.edges():
        solver.Add(x[u, v] == x[v, u])  # Impor que seja binário

    # Limite de tempo de execução (opcional)
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return solver.Objective().Value(), [(u, v) for u, v in G.edges() if x[u, v].solution_value() > 0.5]
    else:
        print('Nenhuma solução ótima encontrada.')
        return None

In [707]:
def transform_to_directed(G):
    """Transforma um grafo não direcionado em um grafo direcionado"""
    directed_G = nx.DiGraph()
    for u, v, data in G.edges(data=True):
        # Adiciona aresta u -> v
        directed_G.add_edge(u, v, **data)
        # Adiciona aresta v -> u
        directed_G.add_edge(v, u, **data)
    return directed_G

In [708]:
# Função de Rótulos nos Vértices com restrições MTZ
def solve_with_mtz(G, d):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Transformar o grafo não direcionado em direcionado
    G = transform_to_directed(G)

    # Variáveis de decisão: 1 se o arco (u, v) estiver na árvore, 0 caso contrário
    y = {}
    for u, v in G.edges():
        y[(u, v)] = solver.BoolVar(f'y_{u}_{v}')

    # Função objetivo: minimizar o custo total das arestas selecionadas
    solver.Minimize(solver.Sum(G[u][v]['weight'] * y[(u, v)] for u, v in G.edges()))

    # Definir um vértice raiz arbitrário (pode ser modificado para qualquer vértice de C)
    root = 0  # A raiz recebe rótulo 0, pode ser central ou definido de outra forma
    labels = {}
    
    # Variáveis de rótulos para os vértices
    for v in G.nodes():
        if v == root:
            labels[v] = solver.IntVar(0, 0, f'label_{v}')  # A raiz recebe rótulo 0
        else:
            labels[v] = solver.IntVar(1, len(G.nodes()), f'label_{v}')  # Demais vértices com rótulos ≥ 1

    # Restrições de rotulação de vértices para evitar ciclos (MTZ)
    for u, v in G.edges():
        if(v != root):
            solver.Add(labels[u] - labels[v] + len(G.nodes()) * y[(u, v)] <= len(G.nodes()) - 1)

    # Restrições de grau mínimo para os vértices centrais
    for v in d:
        solver.Add(solver.Sum(y[(u, v)] for u in G.predecessors(v)) >= d[v])
        solver.Add(solver.Sum(y[(v, j)] for j in G.successors(v)) >= d[v])

    # Restrições de folhas: vértices que não são centrais devem ser folhas (grau 1)
    for v in G.nodes():
        if v not in d:
            solver.Add(solver.Sum(y[(u, v)] for u in G.predecessors(v)) + solver.Sum(y[(v, j)] for j in G.successors(v)) == 1)

    # Definir limite de tempo de execução
    solver.SetTimeLimit(1800000)  # 30 minutos em milissegundos

    # Resolver o problema
    start_time = time.time()
    status = solver.Solve()
    final_time = time.time() - start_time

    # Exibir o status da solução
    if status == pywraplp.Solver.OPTIMAL:
        print("Solução ótima encontrada.")
    elif status == pywraplp.Solver.FEASIBLE:
        print("Solução viável encontrada, mas pode não ser ótima.")
    else:
        print("Solução não encontrada.")

    return final_time

In [709]:
import networkx as nx

def read_graph_from_file(file_path):
    with open(file_path, 'r') as f:
        # Primeira linha: n (vértices), nc (centrais), m (arestas)
        n, nc, m = map(int, f.readline().strip().split())
        
        # Próximas nc linhas: centrais e graus mínimos
        centrais = {}
        for _ in range(nc):
            i, d = map(int, f.readline().strip().split())
            centrais[i] = d
        
        # Próximas m linhas: arestas e custos
        G = nx.Graph()
        for _ in range(m):
            i, j, c = map(int, f.readline().strip().split())
            G.add_edge(i, j, weight=c)
        
    return G, centrais

# Exemplo de uso
file_path = './Instancias/tb8ch4_1.txt'
grafo, centrais = read_graph_from_file(file_path)

# Exibindo o grafo e as informações dos vértices centrais
print("Grafo:", grafo.edges(data=True))
print("Centrais e graus mínimos:", centrais)


Grafo: [(1, 2, {'weight': 676}), (1, 3, {'weight': 535}), (1, 4, {'weight': 355}), (1, 5, {'weight': 405}), (1, 6, {'weight': 650}), (1, 7, {'weight': 266}), (1, 8, {'weight': 306}), (1, 9, {'weight': 320}), (1, 10, {'weight': 389}), (1, 11, {'weight': 485}), (2, 3, {'weight': 277}), (2, 4, {'weight': 491}), (2, 5, {'weight': 408}), (2, 6, {'weight': 30}), (2, 7, {'weight': 555}), (2, 8, {'weight': 484}), (2, 9, {'weight': 361}), (2, 10, {'weight': 410}), (2, 11, {'weight': 234}), (3, 4, {'weight': 243}), (3, 5, {'weight': 461}), (3, 6, {'weight': 273}), (3, 7, {'weight': 330}), (3, 8, {'weight': 259}), (3, 9, {'weight': 306}), (3, 10, {'weight': 452}), (3, 11, {'weight': 323}), (4, 5, {'weight': 483}), (4, 6, {'weight': 477}), (4, 7, {'weight': 98}), (4, 8, {'weight': 54}), (4, 9, {'weight': 299}), (4, 10, {'weight': 468}), (4, 11, {'weight': 424})]
Centrais e graus mínimos: {1: 3, 2: 4, 3: 2, 4: 3}


In [710]:
import time

def compare_formulations(G,d):
    # Suponha que 'd' é um dicionário que você já definiu contendo os graus mínimos dos vértices centrais

    # start_time = time.time()
    # obj_value_subtour = solve_with_subtour_elimination(G, d)
    # time_subtour = time.time() - start_time

    # print(f"Subtour Elimination: Objective Value = {obj_value_subtour}, Time = {time_subtour} seconds")
    aux = transform_to_directed(G)
    obj_value_mtz = solve_with_mtz(aux, d)


    print(f"Rotulos nos vertice Time ={obj_value_mtz} seconds")

# Exemplo de comparação
compare_formulations(grafo, centrais)


Solução não encontrada.
Rotulos nos vertice Time =0.058001041412353516 seconds
